# 问题：
1. 时间怎么计算，over 30 天，算不算record出现的这一天
2. past day是同一天的两种情况，如何赋值，这个record第一次出现，这个record第二第三第四次出现

# learning:
1. copy 可以避免你一直跑data
2. merge可以避免循环dataframe
    -  循环比较可以把count做完

In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv("applications_data.csv")

In [3]:
l = df.columns

In [4]:
for x in l:
    df[x] = df[x].astype(str)

In [5]:
df['zip5']=df['zip5'].apply(lambda x: '{0:0>5}'.format(x))

In [6]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0


In [7]:
# see the mis-filled values for the miscount
df.describe()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
count,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000
unique,1000000,365,835819,78136,177001,828774,26370,42673,28244,2
top,119304,20160816,999999999,EAMSTRMT,ERJSAXA,123 MAIN ST,68138,19070626,9999999999,0
freq,1,2877,16935,12658,8580,1079,823,126568,78512,985607


In [8]:
df["ssn"].replace("9999999999",np.nan,inplace=True)

In [9]:
df['homephone'].replace('9999999999',np.nan,inplace=True)
df['address'].replace('123 MAIN ST',np.nan,inplace=True)
df['dob'].replace('19070626',np.nan,inplace=True)

In [10]:
df['ssn'] = df["ssn"].fillna(df["record"])
df['homephone'] = df['homephone'].fillna(df['record'])
df['address'] = df['address'].fillna(df['record'])
df['dob'] = df['dob'].fillna(df['record'])

In [11]:
# change the date column into the data type
df["date"] = df["date"].astype("int")
df['appyear'] = (df['date'] / 10000).astype('int')
df['appmonth'] = ((df['date'] - df['appyear'] * 10000) / 100).astype('int')
df['appday'] = df['date'] - df['appyear'] * 10000 - df['appmonth'] * 100
df['date'] = pd.to_datetime(df.appyear*10000+df.appmonth*100+df.appday,format='%Y%m%d')
df = df.drop(columns = ['appyear','appmonth','appday'])

## 1. Build Entities
we tried to create as much as entities as possible, so we two two join almost all the field as possible, and we three three three join almost all the field as possible

In [12]:
df["name"] = df["firstname"] + df["lastname"]

In [13]:
coblist = ["ssn","dob","address",'homephone' ,'name','zip5']

In [14]:
len(coblist)

6

In [15]:
for i in range(len(coblist)):
    for j in range(len(coblist)):
        if i<j:
            new_col = coblist[i] + coblist[j]
            df[new_col] = df[coblist[i]] + df[coblist[j]]

In [16]:
for i in range(len(coblist)):
    for j in range(len(coblist)):
        for k in range(len(coblist)):
            if i<j<k:
                new_col = coblist[i] + coblist[j] + coblist[k]
                df[new_col] = df[coblist[i]] + df[coblist[j]] + df[coblist[k]]

In [17]:
df['name-ssn-homephone-dob-address-zip5'] = df['name'] +df['ssn'] + df['homephone']+ df['dob'] + df['address'] + df['zip5']
df['name-ssn-homephone-dob'] = df['name'] +df['ssn'] + df['homephone'] + df['dob'] 
df['name-ssn-address-zip5'] = df['name'] +df['ssn'] + df['address'] + df['zip5'] 

## 2. Calculating count days 


In [18]:
attributes = list(df.columns)

In [19]:
attributes.remove('fraud_label')
attributes.remove('date')
attributes.remove('zip5')
attributes.remove('firstname')
attributes.remove('lastname')
attributes.remove('dob')
attributes.remove('record')

In [20]:
attributes

['ssn',
 'address',
 'homephone',
 'name',
 'ssndob',
 'ssnaddress',
 'ssnhomephone',
 'ssnname',
 'ssnzip5',
 'dobaddress',
 'dobhomephone',
 'dobname',
 'dobzip5',
 'addresshomephone',
 'addressname',
 'addresszip5',
 'homephonename',
 'homephonezip5',
 'namezip5',
 'ssndobaddress',
 'ssndobhomephone',
 'ssndobname',
 'ssndobzip5',
 'ssnaddresshomephone',
 'ssnaddressname',
 'ssnaddresszip5',
 'ssnhomephonename',
 'ssnhomephonezip5',
 'ssnnamezip5',
 'dobaddresshomephone',
 'dobaddressname',
 'dobaddresszip5',
 'dobhomephonename',
 'dobhomephonezip5',
 'dobnamezip5',
 'addresshomephonename',
 'addresshomephonezip5',
 'addressnamezip5',
 'homephonenamezip5',
 'name-ssn-homephone-dob-address-zip5',
 'name-ssn-homephone-dob',
 'name-ssn-address-zip5']

In [21]:
len(attributes)

42

## 3. Counting pastdays


In [22]:
# pastday 是一个只有index的df
pastday = df[['record']].set_index('record').copy()

In [23]:
pastday.head()

""
record
1
2
3
4
5


In [24]:
# for entity in attributes:
entity = "ssn"
data1 = df[['record','date',entity]]
data2 = df[['record','date',entity]]
data1.record = data1.record.astype(int)
data2.record = data2.record.astype(int)
tempdf = pd.merge(data1,data2, left_on = entity, right_on = entity)
tempdf = tempdf[tempdf['record_x'] > tempdf['record_y']] # merge自动加入_x, _y命名
new_col = entity+'_' +'pastday'
tempdf[new_col] = tempdf['date_x'] - tempdf['date_y']
tempdf[new_col] = tempdf[new_col].astype('timedelta64[D]')



//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [25]:
tempdf.head()

,record_x,date_x,ssn,record_y,date_y,ssn_pastday
4,606381,2016-08-10,387482503,2,2016-01-01,222.0
7,743271,2016-09-29,387482503,2,2016-01-01,272.0
8,743271,2016-09-29,387482503,606381,2016-08-10,50.0
14,765456,2016-10-07,24065868,5,2016-01-01,280.0
19,200178,2016-03-14,415812149,7,2016-01-01,73.0


In [26]:
#只保留最近一次发生的pastday number, 每个record只保留一条
tempday = tempdf.groupby('record_x').agg({new_col:'min'})
tempday.index = tempday.index.rename('record')
tempday.index = tempday.index.astype('str')

#第一次发生的pastday值是离2015-12-31的值, 用fillna计算
pastday = pd.merge(pastday, tempday, left_index = True, right_index = True, how = 'left')
data1['initial_date'] = '2015-12-31'
data1['initial_date'] = pd.to_datetime(data1['initial_date'])
data1['filling'] = data1['date'] - data1['initial_date'] 
data1['filling'] = data1['filling'].astype('timedelta64[D]')
data1 = data1.set_index('record')
data1.index = data1.index.astype('str')
pastday[new_col] = pastday[new_col].fillna(data1['filling'])

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [27]:
tempday[tempday.isnull()==True]

,ssn_pastday
record,
23,NaN
65,NaN
69,NaN
75,NaN
284,NaN
...,...
999983,NaN
999984,NaN
999989,NaN


## Count Velocity

In [28]:
import datetime

In [29]:
finaldf = df[['record']].set_index('record').copy()

In [30]:
df['join_ts1'] = df['date']

In [31]:
for t in [0,1,3,7,30,90,180]:
    end_time = 'join_ts2_'+str(t)
    df[end_time] = df['date'] + datetime.timedelta(t)

In [32]:
data3 = df[['record','date', entity]]
data4 = df[['join_ts1','join_ts2_0','join_ts2_1','join_ts2_3','join_ts2_7','join_ts2_30','join_ts2_90','join_ts2_180',entity,'record']]
data3.record = data3.record.astype(int)
data4.record = data4.record.astype(int)
tempdf = pd.merge(data3,data4, left_on = entity, right_on = entity)
# for t in [0,1,3,7,30,90,180]:
#     name_end_time = 'join_ts2_' + str(t)
#     tempdf2 = tempdf[(tempdf['date'] >= tempdf['join_ts1']) & (tempdf['date'] <= tempdf[name_end_time])& (tempdf['record_x']>=tempdf['record_y'])]
#     temp_name = str(entity) + '_' + 'count' + str(t) + '_'
#     tempGroupby = tempdf2[['record_x','date']].groupby('record_x')
#     tempGroupby = getattr(tempGroupby, 'count')().add_prefix(temp_name)
#     tempGroupby.index = tempGroupby.index.rename('record')
#     tempGroupby.index = tempGroupby.index.astype('str')
#     finaldf = pd.merge(finaldf, tempGroupby,left_index = True, right_index = True, how = 'left')

In [ ]:
# 计算count方法1 ： 查看过去的值和现在值的距离是不是所关心的距离
t=30
name_end_time = 'join_ts2_' + str(t)
tempdf2 = tempdf[(tempdf['date'] >= tempdf['join_ts1']) & (tempdf['date'] <= tempdf[name_end_time])& (tempdf['record_x']>=tempdf['record_y'])]
temp_name = str(entity) + '_' + 'count' + str(t) + '_'
tempGroupby = tempdf2[['record_x','date']].groupby('record_x')
tempGroupby = getattr(tempGroupby, 'count')().add_prefix(temp_name)
tempGroupby.index = tempGroupby.index.rename('record')
tempGroupby.index = tempGroupby.index.astype('str')
finaldf = pd.merge(finaldf, tempGroupby,left_index = True, right_index = True, how = 'left')

In [ ]:
# 计算count方法2 ： 查看过去的值是不是在所关心的范围里


In [ ]:
t=30
range_high = tempdf['date']
range_low = tempdf['date']-t
tempdf2 = tempdf[(tempdf['join_ts1'] >= range_low) & (tempdf['date'] <= range_high)& (tempdf['record_x']>=tempdf['record_y'])]
temp_name = str(entity) + '_' + 'count' + str(t) + '_'
tempGroupby = tempdf2[['record_x','date']].groupby('record_x')
# tempGroupby = getattr(tempGroupby, 'count')().add_prefix(temp_name)
#等同于
tempGroupby = tempGroupby.agg({'date':'count'})
tempGroupby.index = tempGroupby.index.rename('record')
tempGroupby.index = tempGroupby.index.astype('str')
finaldf = pd.merge(finaldf, tempGroupby,left_index = True, right_index = True, how = 'left')